## Instalación de librerías Tensorflow y Keras
Previamente debemos de ejecutar los siguientes comandos en Anaconda Prompt:
* 	```conda install tensorflow	```
* 	```conda install keras	```

El análisis se realizara con Keras pero necesitamos Tensorflow ya que Keras esta creada en base a Tensorflow.

## Importando las librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Obteniendo datos
Importamos los datos limpios que han sido tratados anteriormente.

In [2]:
mammographic_masses = pd.read_csv('../processed/mammographic_masses_clean.csv')

In [23]:
mammographic_masses.head(10)

,Age,Shape,Margin,Density,Severity
0,67,3,5,3,1
1,58,4,5,3,1
2,28,1,1,3,0
3,57,1,5,3,1
4,76,1,4,3,1
5,42,2,1,3,1
6,36,3,1,2,0
7,60,2,1,2,0
8,54,1,1,3,0
9,52,3,4,3,0


In [4]:
mammographic_masses.describe()

,Age,Shape,Margin,Density,Severity
count,831.000000,831.000000,831.000000,831.000000,831.000000
mean,55.777377,2.783394,2.814681,2.915764,0.484958
std,14.663528,1.242331,1.566771,0.350737,0.500075
min,18.000000,1.000000,1.000000,1.000000,0.000000
25%,46.000000,2.000000,1.000000,3.000000,0.000000
50%,57.000000,3.000000,3.000000,3.000000,0.000000
75%,66.000000,4.000000,4.000000,3.000000,1.000000
max,96.000000,4.000000,5.000000,4.000000,1.000000


In [5]:
mammographic_masses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 5 columns):
Age         831 non-null int64
Shape       831 non-null int64
Margin      831 non-null int64
Density     831 non-null int64
Severity    831 non-null int64
dtypes: int64(5)
memory usage: 32.5 KB


## Construyendo las matrices
Creamos una **matriz X** con los datos de características (Age, Shape, Margin, Density) y otra **matriz y** con la variable objetivo Severity

In [6]:
X = mammographic_masses.iloc[:,0:4].values
y = mammographic_masses.iloc[:,4].values
X

array([[67,  3,  5,  3],
       [58,  4,  5,  3],
       [28,  1,  1,  3],
       ...,
       [64,  4,  5,  3],
       [66,  4,  5,  3],
       [62,  3,  3,  3]], dtype=int64)

## División en conjunto de entrenamiento y prueba
Reservamos el 75% de los datos para entrenamiento y 25% para pruebas

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

## Estandarización de las variables
StandardScaler es una función de ScikitLearn que nos ayuda a escalar nuestros datos.

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Construyendo la red neuronal
Importamos Keras que es la librería que nos ayuda a crear redes neuronales. Usaremos el modulo Sequential para inicializar la red y el modelo Dense para añadir capas ocultas.

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [11]:
classifier = Sequential()

Añadiremos las capas ocultas utilizando la función Dense. A continuación una breve descripción de los parámetros:
* **output_dim**: es el número de nodos que queremos añadir a la capa.
* **init**: inicialización del descenso de gradiente estocástico, los pesos iniciales serán una variable aleatoria uniforme.
* **input_dim**: indica el número de variables que va a recibir (en este caso 4), solo es necesaria en la primera capa.
* **activation**: cada neurona aplica una función de activación a la suma ponderada de los inputs. Mientras más cercano a 1 sea el valor de esta función la neurona estará más activada y pasará más señal de interacción.

In [12]:
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 4))
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=4, units=6, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.
C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until


Ahora teniendo todas las capas de nuestra red neuronal pasaremos a compilarla usando la función compile.
* **optimizer**: nos indica el método que utilizaremos para seleccionar los pesos óptimos para la red. Usaremos Adam que es un un tipo algoritmo de Descenso de Gradiente Estocástico.
* **loss**: usa la función ‘binary_crossentropy’ que nos sirve para clasificar en 2 categorias.
* **metrics**: mide el desempeño de nuestra red neuronal

In [13]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Hemos creado nuestra red neuronal. Ahora entrenaremos el modelo. Usamos la función fit para buscar los pesos optimos de la red.
* **batch_size**: sirve para especificar el número de observaciones que la red necesita entrenar antes de actualizar los pesos.
* **nb_epoch**: indica el número de iteraciones que realizaremos. 

In [14]:
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
623/623 [==============================] - 3s 5ms/step - loss: 0.6925 - acc: 0.5730
Epoch 2/100
623/623 [==============================] - 0s 96us/step - loss: 0.6782 - acc: 0.6918
Epoch 3/100
623/623 [==============================] - 0s 109us/step - loss: 0.6182 - acc: 0.7994
Epoch 4/100
623/623 [==============================] - 0s 112us/step - loss: 0.5406 - acc: 0.7945
Epoch 5/100
623/623 [==============================] - 0s 104us/step - loss: 0.4936 - acc: 0.8058
Epoch 6/100
623/623 [==============================] - 0s 106us/step - loss: 0.4674 - acc: 0.8058
Epoch 7/100
623/623 [==============================] - 0s 104us/step - loss: 0.4553 - acc: 0.8074
Epoch 8/100
623/623 [==============================] - 0s 119us/step - loss: 0.4504 - acc: 0.8058
Epoch 9/100
623/623 [==============================] - 0s 116us/step - loss: 0.4476 - acc: 0.8026
Epoch 10/100
623/623 [==============================] - 0s 111us/step - loss: 0.4462 - acc: 0.8026
Epoch 11/100
623/623 [

La predicción nos dará la probabilidad de pertenecer a un grupo u otro. Convertiremos esta probabilidad en 0 para las probabilidades menores que 0.5 y 1 para las mayores.

In [21]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

## Evaluación

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[86 25]
 [15 82]]


In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.77      0.81       111
           1       0.77      0.85      0.80        97

   micro avg       0.81      0.81      0.81       208
   macro avg       0.81      0.81      0.81       208
weighted avg       0.81      0.81      0.81       208

